<a href="https://colab.research.google.com/github/MalcolmMashig/CS6501-Information-Retrieval-Project/blob/main/issue_query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Function: `issue_query`

In [2]:
!pip install fake_useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp37-none-any.whl size=13485 sha256=49a9b7a46956f765150998d790f0556ac98d234d2e2cae6c9ec16faad1078497
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
Successfully built fake-useragent


In [1]:
import urllib
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import re
import nltk
nltk.download('words')
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
def issue_query(query, n_results, scrape_websites):
  ua = UserAgent()
  google_url = "https://www.google.com/search?q=" + urllib.parse.quote_plus(query) + "&num=" + str(n_results)
  response = requests.get(google_url, {"User-Agent": ua.random})
  soup = BeautifulSoup(response.text, "html.parser")
  result_div = soup.find_all('div', attrs = {'class': 'ZINbbc'})
  links = []
  titles = []
  descriptions = []
  for r in result_div:
      try:
          link = r.find('a', href = True)
          title = r.find('div', attrs={'class':'vvjwJb'}).get_text()
          description = r.find('div', attrs={'class':'s3v9rd'}).get_text()
          if link != '' and title != '' and description != '': 
              links.append(link['href'])
              titles.append(title)
              descriptions.append(description)
      except:
          continue
  to_remove = []
  clean_links = []
  for i, l in enumerate(links):
      clean = re.search('\/url\?q\=(.*)\&sa',l)
      if clean is None:
          to_remove.append(i)
          continue
      clean_links.append(clean.group(1))
  for x in to_remove:
      del titles[x]
      del descriptions[x]
  if scrape_websites:
    web_texts = []
    words = set(nltk.corpus.words.words())
    stop_words = stopwords.words('english')
    for url in clean_links:
        r = requests.get(url)
        html = r.text
        soup = BeautifulSoup(html, "html5lib")
        text = soup.get_text()
        text = text.replace('\n', ' ')
        new_text = " ".join(w for w in nltk.wordpunct_tokenize(text)
                            if (w.lower() in words)
                            and (w.lower() not in stop_words)
                            and (len(w.lower()) > 1))
        web_texts.append(new_text)
    return clean_links, titles, descriptions, web_texts
  return clean_links, titles, descriptions

In [8]:
# example
issue_query(query = "'columbia'", 
            n_results = 10, 
            scrape_websites = True)
# scraping websites is very slow and should be avoided if possible

(['https://www.columbia.com/',
  'https://www.columbia.edu/',
  'https://www.como.gov/',
  'https://www.columbiasc.net/',
  'https://www.colum.edu/',
  'http://www.ccis.edu/',
  'https://www.experiencecolumbiasc.com/'],
 ['Columbia Sportswear',
  'Columbia University in the City of New York',
  'City of Columbia, Missouri -',
  'Welcome to the City of Columbia',
  'Columbia College Chicago',
  'Columbia College | Online & On-Campus College Classes',
  'Columbia SC: Things to Do, Restaurants, Hotels & Events'],
 ["Best deals yet - 50% off select gear. Now is the time to get those Columbia jackets, pants, shoes and gear you've been needing.MenWomenSaleKidsWomen's SaleMen's Sale",
  'The Columbia Lions football team lifts a teammate into the air in celebration. Athletics · Two graduates wearing regalia\xa0...\nAbout · Admissions · Departments · Careers at Columbia',
  'Browse Categories: City Government · Doing Business in Columbia · Living in Columbia · Visitors · City Services A-Z. CoMo